In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from openeye import oechem
import json
from fragmenter import workflow_api, fragment, torsions, chemi
import cmiles
import oenotebook as oenb
import numpy as np

In [2]:
def get_torsion(mol, rot_bond):
    """
    Find torsion in fragment that corresponds to same rotatable bond in parent molecule
    parameters:
    ----------
    mol: OEMol with map indices to parent molecule
    rot_bond: tuple
        (mapidx_1, mapidx_2)
    return:
    -------
    dihdral: list of dihedral atom indices
    """
    tors = [[tor.a, tor.b, tor.c, tor.d] for tor in oechem.OEGetTorsions(mol)]
    filtered_torsions = torsions.one_torsion_per_rotatable_bond(tors)
    mapped_tors = [[i.GetMapIdx() for i in t] for t in filtered_torsions]
    cbs = [(t[1], t[2]) for t in mapped_tors]
    try:
        dihedral = mapped_tors[cbs.index(rot_bond)]
    except ValueError:
        dihedral = mapped_tors[cbs.index(tuple(reversed(rot_bond)))]
    dihedral = [d-1 for d in dihedral]
    return dihedral



In [3]:
with open('../../combinatorial_fragmentation/filter/filtered_kinase_inhibitors.json', 'r') as f:
    kinase_inhibitors = json.load(f)


In [4]:
frags = {}
for ki in kinase_inhibitors:
    with open('{}/{}_frags_to_drive_2.json'.format(ki, ki), 'r') as f:
        ki_frags = json.load(f)

    for bond in ki_frags:
        b = bond.split('[')[-1].split(']')[0].split(',')
        b = (int(b[0]), int(b[1]))
        for frag_type in ki_frags[bond]:
            for frag, wbo in ki_frags[bond][frag_type]:
                try:
                    mol_ids = cmiles.get_molecule_ids(frag, strict=False)
                except:
                    mol = oechem.OEMol()
                    oechem.OESmilesToMol(mol, frag)
                    mols = fragment._expand_states(mol, enumerate='stereoisomers')
                    mol_ids = []
                    for m in mols:
                        stereo_smiles = oechem.OEMolToSmiles(m)
                        mol_ids.append(cmiles.get_molecule_ids(stereo_smiles, strict=False))
                if not isinstance(mol_ids, list):
                    mol_ids = [mol_ids]
                    
                for mol_id in mol_ids:
                    smiles = mol_id['canonical_isomeric_smiles']
                    if smiles not in frags:
                        frags[smiles] = {'identifiers': mol_id}
                        frags[smiles]['provenance'] = {}
                        frags[smiles]['provenance']['canonicalization'] = mol_id.pop('provenance')
                        frags[smiles]['provenance']['parent_molecule'] = ki
                        frags[smiles]['provenance']['central_rot_bond'] = []
                        frags[smiles]['provenance']['map_to_parent'] = []
                        frags[smiles]['provenance']['wbo'] = []
                    frags[smiles]['provenance']['central_rot_bond'].append(b)
                    frags[smiles]['provenance']['map_to_parent'].append(frag)
                    frags[smiles]['provenance']['wbo'].append(wbo)



/Users/sternc1/src/openforcefield/clean_versions/cmiles/cmiles/_cmiles_oe.py:405: UserWarning: Stereochemistry is unspecified. Problematic atoms [(3, 'N')], problematic bonds [], SMILES: C#CCN1CCC[C@H]1c2nc(c3n2ccnc3N)c4ccc(cc4)C(=O)N
  problematic_bonds, oechem.OEMolToSmiles(molecule)))
/Users/sternc1/src/openforcefield/clean_versions/cmiles/cmiles/_cmiles_oe.py:405: UserWarning: Stereochemistry is unspecified. Problematic atoms [(3, 'N')], problematic bonds [], SMILES: C#CCN1CCC[C@H]1c2nc(c3n2ccnc3N)c4ccc(cc4)C(=O)Nc5ccccn5
  problematic_bonds, oechem.OEMolToSmiles(molecule)))
/Users/sternc1/src/openforcefield/clean_versions/cmiles/cmiles/_cmiles_oe.py:405: UserWarning: Stereochemistry is unspecified. Problematic atoms [(30, 'N')], problematic bonds [], SMILES: C#CCN1CCC[C@H]1c2nc(c3n2ccnc3N)c4ccc(cc4)C(=O)Nc5ccccn5
  problematic_bonds, oechem.OEMolToSmiles(molecule)))
/Users/sternc1/src/openforcefield/clean_versions/cmiles/cmiles/_cmiles_oe.py:405: UserWarning: Stereochemistry is un

/Users/sternc1/src/openforcefield/clean_versions/cmiles/cmiles/_cmiles_oe.py:405: UserWarning: Stereochemistry is unspecified. Problematic atoms [(20, 'N')], problematic bonds [], SMILES: c1cc(ncc1N2CCNCC2)Nc3ncc4c(n3)N(CC=C4)C5CCCC5
  problematic_bonds, oechem.OEMolToSmiles(molecule)))
/Users/sternc1/src/openforcefield/clean_versions/cmiles/cmiles/_cmiles_oe.py:405: UserWarning: Stereochemistry is unspecified. Problematic atoms [(20, 'N')], problematic bonds [], SMILES: CC1=CCN(c2c1cnc(n2)Nc3ccc(cn3)N4CCNCC4)C5CCCC5
  problematic_bonds, oechem.OEMolToSmiles(molecule)))
/Users/sternc1/src/openforcefield/clean_versions/cmiles/cmiles/_cmiles_oe.py:405: UserWarning: Stereochemistry is unspecified. Problematic atoms [(14, 'N')], problematic bonds [], SMILES: c1ccnc(c1)Nc2ncc3c(n2)N(CC=C3)C4CCCC4
  problematic_bonds, oechem.OEMolToSmiles(molecule)))
/Users/sternc1/src/openforcefield/clean_versions/cmiles/cmiles/_cmiles_oe.py:405: UserWarning: Stereochemistry is unspecified. Problematic atom

In [5]:
# Save all jobs
with open('all_frags_to_drive.json', 'w') as f:
    json.dump(frags, f, indent=2, sort_keys=True)

In [6]:
# Only run fragments where the WBO changes more than 0.1
to_run = {}
for ki in kinase_inhibitors:
    print(ki)
    with open('{}/{}_frags_to_drive_2.json'.format(ki, ki), 'r') as f:
        ki_frags = json.load(f)
    for bond in ki_frags:
        wbos = []
        for frag_type in ki_frags[bond]:
            for frag, wbo in ki_frags[bond][frag_type]:
                wbos.append((wbo, frag))
        sorted_wbo = sorted(wbos)
        diff = sorted_wbo[-1][0] - sorted_wbo[0][0]
        if diff > 0.1:
            if ki not in to_run:
                to_run[ki] = {}
            # Keep frags with largest difference, smallest and largest fragment
            to_run[ki][bond] = [wbos[0], wbos[-1], sorted_wbo[0], sorted_wbo[-1]]

Abemaciclib
Acalabrutinib
Axitinib
Baricitinib
Binimetinib
Brigatinib
Cobimetinib
Crizotinib
Dabrafenib
Dacomitinib
Dasatinib
Gefitinib
Ibrutinib
Idelalisib
Imatinib
Larotrectinib
Lenvatinib
Nilotinib
Palbociclib
Pazopanib
Ponatinib
Ribociclib
Ruxolitinib
Sunitinib
Tofacitinib
Trametinib
Vemurafenib


In [7]:
# Visualize fragments to run 
for ki in to_run:
    for bond in to_run[ki]:
        des_bond = tuple(int(i) for i in bond.split('[')[-1].split(']')[0].split(','))
        frags = [f[-1] for f in to_run[ki][bond]]
        bos = [f[0] for f in to_run[ki][bond]]
        fname = '{}_bond_{}_{}_to_run.pdf'.format(ki, des_bond[0], des_bond[1])
        chemi.to_pdf(frags, bond_map_idx=des_bond, bo=bos, oname=fname, rows=3, cols=2)

In [8]:
# Prepare input SMILES and provenance only for Idelalisib because it's the only bond that is not directly between 2 rings
filtered_frags = {}
for ki in to_run:
    if ki != 'Idelalisib':
        continue
    for b in to_run[ki]:
        for wbo, frag in to_run[ki][b]:
            try:
                mol_ids = cmiles.get_molecule_ids(frag, strict=False)
            except:
                mol = oechem.OEMol()
                oechem.OESmilesToMol(mol, frag)
                mols = fragment._expand_states(mol, enumerate='stereoisomers')
                mol_ids = []
                for m in mols:
                    stereo_smiles = oechem.OEMolToSmiles(m)
                    mol_ids.append(cmiles.get_molecule_ids(stereo_smiles, strict=False))
            if not isinstance(mol_ids, list):
                mol_ids = [mol_ids]

            for mol_id in mol_ids:
                smiles = mol_id['canonical_isomeric_smiles']
                if smiles not in filtered_frags:
                    filtered_frags[smiles] = {'identifiers': mol_id}
                    filtered_frags[smiles]['provenance'] = {}
                    filtered_frags[smiles]['provenance']['canonicalization'] = mol_id.pop('provenance')
                    filtered_frags[smiles]['provenance']['parent_molecule'] = ki
                    filtered_frags[smiles]['provenance']['central_rot_bond'] = []
                    filtered_frags[smiles]['provenance']['map_to_parent'] = []
                    filtered_frags[smiles]['provenance']['wbo'] = []
                filtered_frags[smiles]['provenance']['central_rot_bond'].append(b)
                filtered_frags[smiles]['provenance']['map_to_parent'].append(frag)
                filtered_frags[smiles]['provenance']['wbo'].append(wbo)

# Save Idelalisib jobs
with open('Idelalisib_frags.json', 'w') as f:
    json.dump(filtered_frags, f, indent=2, sort_keys=True)